# Обучение рекуррентной нейронной сети для предсказания следующей фонемы
## В данном коде мы будем обучать базовую рекуррентную нейронную сеть с прямым и обратным распространением

In [4]:
# импорт библиотек
import sys
import numpy as np 
from collections import Counter
import random
import math

In [5]:
# чтение датасета, состоящего из фонетически транскрибированных слов (каждая фонема через пробел)
f = open('phon.txt','r')
raw = f.readlines()
f.close()

# создание пустого списка для токенов
tokens = list()
for line in raw[0:]:  
    tokens.append(line.lower().replace("\n","").split(" ")[0:]) #добавление слов в список токенов

# таким образом токены состоят из списков, содержащих слово в транскрипции 
print(tokens[0:5]) #вывод первых пяти элементов (токенов)
print(len(tokens)) #длина списка токенов

[['j', 'u', "r'", 'i'], ['t', "r'", 'i', 'f', 'a', 'n', 'a', 'f'], ['a', "b'", "m'", 'e', 'n'], ['y', 'j', 'u', "l'", 'e'], ['m', 'a', 'c']]
75235


# Формируем список существующих элементов из токенов и присваиваем им уникальный индекс

In [6]:
vocab = set() #создается простой список фонем из словаря
for sound in tokens: #итерация по каждому элементу в списке 'tokens'
    for phon in sound: #теперь по каждому звуку
        if phon == '': #проверям не пуста ли фонема, если да пропускаем
            pass
        else:
            vocab.add(phon) #добавляем фонему в 'vocab', если фонема уже есть, добавление не происходит
vocab = list(vocab) #преобразование множества в список
phon2index = {} #создание пустого словаря для хранения фонемы и ее индекса
for i,phon in enumerate(vocab):  #итерация по списку 'vocab', получая индекс i и фонему 'phon' 
    phon2index[phon]=i
    
def phons2indices(word): 
    '''Функция для преобразования списка Фонем в список индексов'''
    idx = list() #список для индексов
    for phon in word: #проходим по фонемам в слове
        idx.append(phon2index[phon]) #ищем индекс фонемы в словаре 'phon2index' и добавляет его в список 'idx'
    return idx

def softmax(x): 
    '''Функция активации softmax для предсказания следующей фонемы'''
    e_x = np.exp(x - np.max(x)) #ищем экспоненту каждого элемента вектора, вычитая максимальное значение 
    return e_x / e_x.sum(axis=0) #нормализуем экспоненциальные значения, деля на их сумму
    # возвращает вектор вероятностей

# выводим наш список vocab со всеми упомянутыми фонемами
print(vocab)
# длина списка vocab
print(len(vocab))
# выводим словарь, где каждой фонеме присвоен уникальный индекс (начиная с 0)
print(phon2index)
# длина словаря
print(len(phon2index))

['g', 't', 's', 'l', "zh'", "dz'", "sh'", 'p', 'e', 'n', 'y', "f'", 'j', 'b', "l'", "b'", 'dzh', "m'", 'ch_', 'k', 'u', 'i', 'z', 'c', "r'", 'm', 'zh', "v'", 'sh', "j'", "t'", 'sch', "n'", "z'", 'ch', "c'", 'a', 'f', 'r', "s'", 'sc', 'o', "k'", 'h', "p'", "d'", 'v', "h'", "g'", 'd']
50
{'g': 0, 't': 1, 's': 2, 'l': 3, "zh'": 4, "dz'": 5, "sh'": 6, 'p': 7, 'e': 8, 'n': 9, 'y': 10, "f'": 11, 'j': 12, 'b': 13, "l'": 14, "b'": 15, 'dzh': 16, "m'": 17, 'ch_': 18, 'k': 19, 'u': 20, 'i': 21, 'z': 22, 'c': 23, "r'": 24, 'm': 25, 'zh': 26, "v'": 27, 'sh': 28, "j'": 29, "t'": 30, 'sch': 31, "n'": 32, "z'": 33, 'ch': 34, "c'": 35, 'a': 36, 'f': 37, 'r': 38, "s'": 39, 'sc': 40, 'o': 41, "k'": 42, 'h': 43, "p'": 44, "d'": 45, 'v': 46, "h'": 47, "g'": 48, 'd': 49}
50


# Матрицы для предсказания

In [92]:
embed_size = 15 #размерность векторного представления фонем, то есть фонем из 50 элементов
embed = (np.random.rand(len(vocab),embed_size) + 0.005) #векторов представлены случайными числами в диапазоне от 0 до 1 с плавающей точкой
recurrent = np.eye(embed_size) #рекуррентная матрица (первоначально единичная)
start = np.zeros(embed_size) #начальное векторное представление фонемы именно так начинается моделирование предлопредсказаний в нейронных сетях
decoder = (np.random.rand(embed_size, len(vocab)) + 0.005) #создается весовая матрица decoder для преобразования векторного представления фонемы в вектор вероятностей
one_hot = np.eye(len(vocab)) #вспомогательная (единичная) матрица для расчета функции потерь

print('Вектор эмбейдинга:' + '\n', embed[:5])
print('Матрица декодера:'  + '\n', decoder[:5])

Вектор эмбейдинга:
 [[0.22599767 0.21025573 0.619817   0.10181126 0.82417756 0.35229479
  0.17420087 0.89946999 0.47737445 0.48795129 0.53865006 0.58937165
  0.26483124 0.98432685 0.59957953]
 [0.02251275 0.64788816 0.48913721 0.32254489 0.66841533 0.26877149
  0.09538665 0.61338554 0.26914206 0.73395576 0.14828368 0.16010576
  1.00151956 0.53834975 0.0566615 ]
 [0.32805772 0.92629265 0.59167864 0.1310873  0.66598231 1.00058382
  0.71412372 0.80119982 0.96412475 0.43115836 0.3856158  0.0533974
  0.48260591 0.30240927 0.65657666]
 [0.58408365 0.33831098 0.92089239 0.7358892  0.55880837 0.12016441
  0.384233   0.64509949 0.42492332 0.67311826 0.38499143 0.63442764
  0.74707303 0.62388644 0.20646057]
 [0.63428399 0.70506957 0.07214126 0.97559851 0.56698691 0.24504028
  0.90514839 0.66933127 0.54624535 0.9113055  0.88417847 0.88768452
  0.07196143 0.18350256 0.69507373]]
Матрица декодера:
 [[0.45525961 0.83517141 0.59554007 0.15524546 0.51143109 0.84368874
  0.42204687 0.49355248 0.8589744

# Функция предсказания

In [93]:
def predict(word):
    '''Функция принимает на вход список индексов фонем (слова)'''    
    layers = list() #Список с именем layers для прямого распространения
    layer = {} #словарб для слоёв
    layer['hidden'] = start #скрытое состояние первого слоя начальным вектором 'start'
    layers.append(layer) #добавляем первый слой в список
    loss = 0 #для хранения суммарной ошибки

    # итерация по слову
    for target_i in range(len(word)): #проходим по всем фонемам в слове
        layer = {} #словарь для этого слоя 
        layer['pred'] = softmax(layers[-1]['hidden'].dot(decoder)) #извлекает вероятность, которую сеть предсказала для правильного следующего слова  

        # вычисляем ошибку для текущего предсказания, используя отрицательный логарифм вероятности правильного слова, добавляем ошибку к общей суммарной ошибке 'loss'
        epsilon = 1e-15 #добавляем маленькое число, которое используется для предотвращения взятия логарифма от нуля
        loss += -np.log(layer['pred'][word[target_i]] + epsilon) #чем ниже вероятность предсказанного слова, тем выше значение loss
        # выисляем скрытое состояние для следующего слоя, используя рекуррентную связь и эмбеддинги текущего слова
        layer['hidden'] = layers[-1]['hidden'].dot(recurrent) + embed[word[target_i]]
        layers.append(layer)
    return layers, loss #возвращаем список слоев 'layers' и суммарную ошибку 'loss'

# Обучение модели по итерациям 

In [ ]:
for iter in range(500000):
    # прямое распространение
    alpha = 0.000001 #сокрость обучения
    phon_tokens = tokens[iter%len(tokens)]
    phon_tokens = [token for token in phon_tokens if token] #отфильтровываем пустые строки
    word = phons2indices(phon_tokens) #выбираем случайное предложение из списка токенов и преобразуем его в список индексов
    layers,loss = predict(word) #получаем список слоев и ошибку

    # обратное распространение для обновления весов 
    for layer_idx in reversed(range(len(layers))):
        layer = layers[layer_idx] 
        target = word[layer_idx-1]
        # проверяем, что слой не первый 
        if(layer_idx > 0): 
            layer['output_delta'] = layer['pred'] - one_hot[target] #вычисляем разницу между предсказанным и ожидаемым вектором
            new_hidden_delta = layer['output_delta'].dot(decoder.transpose()) #вычисляем ошибку скрытого состояния текущего слоя

            # проверяем, что слой не последний
            if(layer_idx == len(layers)-1):
                layer['hidden_delta'] = new_hidden_delta
            else:
                # ошибка скрытого состояния вычисляется на основе ошибки текущего слоя и ошибки скрытого состояния следующего слоя
                layer['hidden_delta'] = new_hidden_delta + layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose())
        else: #если слой первый 
            # вычисляем ошибку скрытого состояния первого слоя на основе ошибки скрытого состояния второго слоя
            layer['hidden_delta'] = layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose())

    # теперь корректируем веса
    # обновляем начальное скрытое состояние на основе ошибки скрытого состояния первого слоя, деля на длину предложения для нормализации
    start -= layers[0]['hidden_delta'] * alpha / (len(word))
    # итерация по всем слоям со второго 
    for layer_idx,layer in enumerate(layers[1:]):
        # корректируем матрицу декодера с учетом скрытого состояния текущего слоя и ошибки выходного слоя, деля на длину предложения для нормализации
        decoder -= np.outer(layers[layer_idx]['hidden'], layer['output_delta']) * alpha / (len(word))
        
        # извлекаем индекс текущего слова из списка индексов предложения
        embed_idx = word[layer_idx]
        embed[embed_idx] -= layers[layer_idx]['hidden_delta'] * alpha / (len(word)) #корректируем вектор эмбеддинга текущего слова с помощью ошибки скрытого состояния текущего слоя, деля на длину предложения для нормализации
        recurrent -= np.outer(layers[layer_idx]['hidden'], layer['hidden_delta']) * alpha / (len(word)) #обновляем матрицу рекуррентной связи с помощью скрытого состояния текущего слоя и ошибки скрытого состояния текущего слоя, деля на длину предложения для нормализации
    
    # выводим перплексию каждые 1000 итераций 
    if(iter % 10000 == 0):
        if str(np.exp(loss/len(word))) == 'nan':
               break
        else:
            print("Perplexity:", iter, str(np.exp(loss/len(word))))

Perplexity: 0 64.20848959499746
Perplexity: 10000 35.622687578403045
Perplexity: 20000 41.58681125677364
Perplexity: 30000 51.72054945250671
Perplexity: 40000 41.42629307153692
Perplexity: 50000 64.7735207636436
Perplexity: 60000 34.41994125714539
Perplexity: 70000 27.494414155269908
Perplexity: 80000 27.27657285561164
Perplexity: 90000 61.002870582512124
Perplexity: 100000 27.77220923832629
Perplexity: 110000 37.719943030984595
Perplexity: 120000 44.651219212438725
Perplexity: 130000 38.69872645664222
Perplexity: 140000 31.806734745628912
Perplexity: 150000 32.1935125889175
Perplexity: 160000 21.874679964333303
Perplexity: 170000 31.841787596784613
Perplexity: 180000 47.786300606911624
Perplexity: 190000 50.49641032155704
Perplexity: 200000 31.49415766486017
Perplexity: 210000 70.7798033631651
Perplexity: 220000 44.46746652470285
Perplexity: 230000 32.37675936483693
Perplexity: 240000 26.988021600040074
Perplexity: 250000 31.57058102123863
Perplexity: 260000 49.75325840843228
Perplexi

# Попросим сеть предсказать следующую фонему в слове с индексом n

In [110]:
word_index = 25

#  l - список слоев, _ - игнорируемая переменная loss
l,_ = predict(phons2indices(tokens[word_index]))
# выводим предложение с индексом 10 в виде списка
print(tokens[word_index])
# итерация по слоям кроме первого и последнего
for i,each_layer in enumerate(l[1:-1]):
    # извлекаем слово из предложения
    input = tokens[word_index][i]
    # извлекаем оригинальное следующее слово предложения
    true = tokens[word_index][i+1]
    # предсказанное следующее слово
    pred = vocab[each_layer['pred'].argmax()]
    # выводим результат
    print("Prev Input:" + input + (' ' * (12 - len(input))) +\
          "True:" + true + (" " * (15 - len(true))) + "Pred:" + pred)

['p', 'a', 't', "v'", 'e', 'r', "d'", 'i', 'l', 'a', "s'"]
Prev Input:p           True:a              Pred:a
Prev Input:a           True:t              Pred:e
Prev Input:t           True:v'             Pred:e
Prev Input:v'          True:e              Pred:l
Prev Input:e           True:r              Pred:a
Prev Input:r           True:d'             Pred:a
Prev Input:d'          True:i              Pred:a
Prev Input:i           True:l              Pred:a
Prev Input:l           True:a              Pred:a
Prev Input:a           True:s'             Pred:a


In [107]:
word_index = 15

#  l - список слоев, _ - игнорируемая переменная loss
l,_ = predict(phons2indices(tokens[word_index]))
# выводим предложение с индексом 10 в виде списка
print(tokens[word_index])
# итерация по слоям кроме первого и последнего
for i,each_layer in enumerate(l[1:-1]):
    # извлекаем слово из предложения
    input = tokens[word_index][i]
    # извлекаем оригинальное следующее слово предложения
    true = tokens[word_index][i+1]
    # предсказанное следующее слово
    pred = vocab[each_layer['pred'].argmax()]
    # выводим результат
    print("Prev Input:" + input + (' ' * (12 - len(input))) +\
          "True:" + true + (" " * (15 - len(true))) + "Pred:" + pred)

['p', 'r', 'a', "l'", 'i', 'zh', 'e', 'l', 'a']
Prev Input:p           True:r              Pred:a
Prev Input:r           True:a              Pred:e
Prev Input:a           True:l'             Pred:e
Prev Input:l'          True:i              Pred:e
Prev Input:i           True:zh             Pred:a
Prev Input:zh          True:e              Pred:a
Prev Input:e           True:l              Pred:a
Prev Input:l           True:a              Pred:i


### Как показывает результат сеть чаще предсказывает гласную /a/, либо любую другую гласную, так как они чаще встречаются в обучающем наборе